install required packages

In [1]:
using Pkg;
Pkg.update();
Pkg.add("DataArrays")

  Updating registry at `~/julia_depots/registries/BGIRegistries`
  Updating git-repo `https://git.bgc-jena.mpg.de/fgans/BGIRegistries`
  Updating registry at `~/julia_depots/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/gdkrmr/WeightedOnlineStats.jl`
  Updating git-repo `https://github.com/milanflach/MultivariateAnomalies.jl.git`
 Resolving package versions...
  Updating `~/julia_depots/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/julia_depots/environments/v1.1/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/julia_depots/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/julia_depots/environments/v1.1/Manifest.toml`
 [no changes]


loading packages

In [2]:
using Pkg;
using ESDL;
using DelimitedFiles;
using Statistics;
#using DataArrays;

load the data cube

In [3]:
c = Cube("/home/jovyan/work/datacube/ESDCv2.0.0/esdc-8d-0.083deg-184x270x270-2.0.0.zarr/")

Collection of ZArray Cube with the following dimensions
Lon                 Axis with 4320 Elements from -179.95833 to 179.9569
Lat                 Axis with 2160 Elements from 89.958336 to -89.95761
Time                Axis with 1702 Elements from 1980-01-05 to 2016-12-30
Variable            Axis with 75 elements: evaporative_stress free_msr_lrt .. lrt_c burnt_area 
Total size: 5.42 TB


subset the cube concerning time and location but keep all variables

In [4]:
d = subsetcube(c, Lon = (73.5, 104.75), 
                  Lat = (25.92, 39.83), 
                  time = (Date(2010,1,1), Date(2014,12,31))
    )

Collection of ZArray Cube with the following dimensions
Lon                 Axis with 375 Elements from 73.54066 to 104.7072
Lat                 Axis with 167 Elements from 39.79187 to 25.958591
Time                Axis with 229 Elements from 2010-01-05 to 2014-12-23
Variable            Axis with 75 elements: evaporative_stress free_msr_lrt .. lrt_c burnt_area 
Total size: 5.01 GB


subset the cube again to drop the variables that contain many missing values (I checked this manually below)

In [5]:
vars = [1,2,4,5,6,7,8,9,10,11,14,15,16,17,18,19,20,21,22,24,26,27,28,29,30,31,32,33,35,36,37,38,39,40,41,43,44,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,70,71,72,73,74,75]
d2 = subsetcube(d,variable=CartesianIndex.(vars))

Collection of ZArray Cube with the following dimensions
Lon                 Axis with 375 Elements from 73.54066 to 104.7072
Lat                 Axis with 167 Elements from 39.79187 to 25.958591
Time                Axis with 229 Elements from 2010-01-05 to 2014-12-23
Variable            Axis with 66 elements: evaporative_stress free_msr_lrt .. lrt_c burnt_area 
Total size: 4.41 GB


fill gaps to have a cube without missing values (otherwise there are problems in the code below)

In [ ]:
d4 = gapFillMSC(d2)

calculate mean value for all variables and the full time period considered

In [ ]:
d3 = mapslices(mean ∘ skipmissing, d4, dims=("Time"))

In [ ]:
d3

save datacube to file, to avoid re-processing of the cube every time the notebook is reloaded

In [ ]:
saveCube(d3, "/home/jovyan/work/workspace/fabis_cube_gap_filled.zarr")

#### next time, start the process from here (or at least skip the mapslices step) ####

In [ ]:
loadCube("/home/jovyan/work/workspace/fabis_cube_gap_filled.zarr")

store the variable names into an array

In [ ]:
vars = getAxis("Variable", d3)

have a look at the variable names

In [ ]:
vars.values

extract longitude and latitude values of the cube grid

In [ ]:
lon = getAxis("Lon", d)

In [ ]:
lat = getAxis("Lat", d)

have a look at the lat and lon values

In [ ]:
lat.values

In [ ]:
lon.values

create dummy array with number of entries matching the total number of grid cells

In [ ]:
col1 = collect(1:length(lon.values)*length(lat.values))
col1 = col1*1.0

create empty matrix with several colums matching the length of total number of grid cells

In [ ]:
mat = [col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1]

create a sequence marking the starting points of each latitude column

In [ ]:
len = length(lat)
seq=[1:len:length(col1);]
append!(seq, length(col1))

write lat and lon value-combinations into the first two columns of the empty matrix

In [ ]:
i = 1
for j in lon.values
  for j1 in lat.values
            mat[i,1:2] = [j j1]
            i = i + 1
        end
    end

In [ ]:
mat

now write the actual values of the data variables into the remaining columns - so far still a problem with missing values; either fix this above by subsetting or interpolating the cube; alternatively replace missing values within the loop (no idea yet how to do this)

the last part has to be out of the loop due to the way it is currently coded...

In [ ]:
for j2 in 1:(length(lon.values)-1)
   
    mat[(seq[j2]:(seq[j2+1]-1)),3:68] = d3[j2,:,1:66]
    print(j2)
        
end

mat[(seq[(length(seq)-1)]:(seq[length(seq)])),3:68] = d3[length(lon.values),:,1,1:66]

In [ ]:
mat[:,1:6]

now load the reference data extracted using the grid developed in Script 1 and build an empirical model

In [ ]:
import Pkg;
Pkg.add("DecisionTree");
Pkg.add("RDatasets")
Pkg.add("ScikitLearn")
#Pkg.add("Koala")

In [ ]:
ls_data = readdlm("ESDL_grid_LS.csv", header=true);
#ls_data2 = ls_data[1];

In [ ]:
mod_data = readdlm("ESDL_grid_MODIS.csv", header=true);
#mod_data2 = mod_data[1];

load additional predictors not available in the ESDL cube

In [ ]:
topo_data = readdlm("topo_mean.txt", header=true);
#topo_data2 = topo_data[1];

In [ ]:
all_data = hcat(mod_data[1][:,:], ls_data[1][:,:], topo_data[1][:,:])

In [ ]:
Pkg.add("DataFrames");
using DataFrames, Serialization, Missings

In [ ]:
all_data2 = convert(DataFrame, all_data);

In [ ]:
all_data3 = DataFrame(colwise(x -> recode(x, "missing"=>missing), all_data2), names(all_data2))

In [ ]:
function rowswith( df::AbstractDataFrame, needle )::Vector{Bool}
        hasit= Vector{Bool}(undef, nrow(df))
	for r in 1:nrow(df)
		hasit[r]= false
        	for c in 1:ncol(df)
			(ismissing(df[r,c]) && ismissing(needle)) && (hasit[r]= true; break)
			(ismissing(needle)) && (continue)
			(ismissing(df[r,c])) && (continue)
			(isnan(df[r,c]) && isnan(needle)) && (hasit[r]= true; break)
			(isnan(needle)) && (continue)
			(isnan(df[r,c])) && (continue)
                        (df[r,c] ==  needle) &&  (hasit[r]= true; break)
                end#for c#
        end#for r#
        hasit
end;##function##


In [ ]:
anymissing= rowswith( all_data3, missing )

In [ ]:
completecases(all_data3) == .!anymissing

In [ ]:
all_data4 = all_data3[ .!anymissing, : ]

In [ ]:
all_data5 = convert(Matrix, all_data4)
all_data6 = Array{Float64}(all_data5)

run random forest

In [ ]:
using ScikitLearn: fit!, predict
using DecisionTree
#using RDatasets
#using Koala


In [ ]:
# train regression forest, using 
# 2 random features
# 10 trees
# averaging of 5 samples per leaf
# and 0.7 portion of samples per tree
random_feat = 5
trees = 500
avg_sampl = 20
portion = 0.7
model = build_forest(all_data6[:,1], all_data6[:,7:19], random_feat, trees, portion, avg_sampl)
# apply learned model
# apply_forest(model, [-0.9,3.0,5.1,1.9,0.0])
# run 3-fold cross validation on regression forest, using 2 random features per split
n_subfeatures=2; n_folds=3
r2 = nfoldCV_forest(all_data6[:,1], all_data6[:,7:19], n_folds, n_subfeatures)
